# PyCity Schools Analysis

- Executive Summary:
  This analysis takes a look at data from 15 different schools and attempts to evaluate 
  the performance of the schools based on the following data categorizations:
  
      - Highest performing schools based on the overall passing percentage.
      - Loweset performing schools based on the overall passing percentage.
      - Both math and reading scores by grade level.
      - And, math and reading scores based on each each school's spending, size, and school type. 

  The first dataframe titled, 'schools_data_complete,' is produced by merging two datasets, and subsequently, 
  various variables and dataframes are created using this dataframe for the analysis.
 
- Findings:
  Upon analysis of the dataframes that were created, as a whole, charter schools outperformed district schools when
  looking at the average percentage of 'overall passing' — calculated by dividing the number of students that passed 
  BOTH math and reading by the total number of students. In fact, when sorting the schools based on their respective
  'overall passing' rates, the top five schools were charter schools, while the bottom five were district schools.
  
  Additionally, schools that were categorized as small(i.e., less than 1000 students) or medium(i.e. between 1000-2000
  students) considerably outperformed large schools (i.e., 2000-5000 students) in 'overall passing.' Furthermore, 
  and contrary to the assumption that increased spending will increase school performance, schools that spent less
  than 585(USD) and between 585-630(USD) performed better than schools that spent more per student. 
  
  

In [1]:
# Importing necessary modules.
import pandas as pd
from pathlib import Path

In [2]:
# Creating file paths.
schools_csv_path = Path("../Resources/schools_complete.csv")
students_csv_path = Path("../Resources/students_complete.csv")

# Reading the files. 
schools_csv_df = pd.read_csv(schools_csv_path)
students_csv_df = pd.read_csv(students_csv_path)

# Merging the two DataFrames and displaying the output.
schools_data_complete = pd.merge(students_csv_df, schools_csv_df, how='left', on="school_name" )
schools_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


# District Summary

In [3]:
# Counting the number of unique schools.
school_count = len(schools_data_complete["school_name"].unique())

In [4]:
# Counting the number of total students.
student_count = schools_data_complete["student_name"].count()

In [5]:
# Finding the total budget, by adding the budgets for each school.
total_budget = schools_data_complete["budget"].unique().sum()

In [6]:
# Calculating the average math score.
avg_math_score = schools_data_complete["math_score"].mean()

# Calculating the average reading score.
avg_reading_score = schools_data_complete["reading_score"].mean()

In [7]:
# Using a conditional to calculate the percentage of students that passed math. 
passing_math = schools_data_complete.loc[(schools_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percent = (passing_math / student_count) * 100

# Using a conditional to calculate the percentage of students that passed reading. 
passing_reading = schools_data_complete.loc[(schools_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percent = (passing_reading / student_count) * 100

In [8]:
# Using two conditional statements to calculate the percentage of students that passed both math AND reading.
overall_passing = schools_data_complete.loc[(schools_data_complete["reading_score"] >= 70) & 
                                           (schools_data_complete["math_score"] >= 70)].count()["student_name"]
overall_passing_percent = (overall_passing / student_count) * 100

In [9]:
# Creating a new DataFrame from the new variables. 
district_summary = pd.DataFrame({"Total Schools": [school_count],
                                "Total Students": [student_count],
                                "Total Budget": [total_budget], 
                                "Average Math Score": [avg_math_score],
                                "Average Reading Score": [avg_reading_score],
                                "% Passing Math": [passing_math_percent],
                                "% Passing Reading": [passing_reading_percent],
                                "% Overall Passing %": [overall_passing_percent]})

# Formatting 
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Displaying the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing %
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


# School Summary

In [10]:
# Creating a variable that groups the 'schools_data_complete' DataFrame by the 'school_name' column.
grouped_school = schools_data_complete.groupby(["school_name"])

In [11]:
# Accessing the 'type' column and using the .first() method to find the school type for each school.
school_types = grouped_school["type"].first()

In [12]:
# Counting the number of students per school. 
per_school_counts = schools_data_complete["school_name"].value_counts()

In [13]:
# Accessing the 'type' column and using the .first() method to find the budget for each school.
per_school_budget = grouped_school["budget"].first()

In [14]:
# Calculating the budget per student for each school by dividing the two previous variables. 
per_school_capita = (per_school_budget) / (per_school_counts)

In [15]:
# Calculating the average math score for each school.
per_school_math = grouped_school["math_score"].mean()

# Calculating the average reading score for each school.
per_school_reading = grouped_school["reading_score"].mean()

In [16]:
# Using a conditional to calculate the number of students that passed math in each school.
students_passing_math = schools_data_complete[schools_data_complete["math_score"] >= 70]
school_students_passing_math = students_passing_math.groupby("school_name").size()

# Using a conditional to calculate the number of students that passed reading in each school.
students_passing_reading = schools_data_complete[schools_data_complete["reading_score"] >= 70]
school_students_passing_reading = students_passing_reading.groupby("school_name").size()

In [17]:
# Using two conditionals to calculate the number of students that passed both math AND reading for eech school. 
students_passing_math_and_reading = schools_data_complete[(schools_data_complete["math_score"] >= 70)
                                                         & (schools_data_complete["reading_score"] >= 70)]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby("school_name").size()

In [18]:
# Calculating the percentage of students that passed math, reading, and both.
per_school_passing_math = school_students_passing_math / per_school_counts * 100
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100

In [19]:
# Creating a new DataFrame from the variables. 
school_summary = pd.DataFrame({"School Type": school_types,
                              "Total Students": per_school_counts,
                              "Total School Budget": per_school_budget,
                              "Per Student Budget": per_school_capita,
                              "Average Math Score": per_school_math,
                              "Average Reading Score": per_school_reading,
                              "% Passing Math": per_school_passing_math,
                              "% Passing Reading": per_school_passing_reading,
                              "% Overall Passing":overall_passing_rate })

# Formatting the DataFrame 
school_summary.index.name = None
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,.2f}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,.2f}".format)

# Displaying the DataFrame
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


# Highest-Performing Schools by (Overall Passing)

In [20]:
# Sorting the 'school_summary' DataFrame by '% Overall Passing' and displaying the top five schools. 
top_schools = school_summary.sort_values(by='% Overall Passing', ascending=False)
top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


# Lowest-Performing Schools by (Overall Passing %)

In [21]:
# Sorting the 'school_summary' DataFrame by '% Overall Passing' and displaying the bottom five schools. 
bottom_schools = school_summary.sort_values(by='% Overall Passing', ascending=True)
bottom_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


# Math Scores by Grade

In [22]:
# Tapping into the 'schools_data_complete' DataFrame, set 'grade' to 9th. 
# Group the initial variable by 'school_name' and calculate the average of 'math_score'

ninth_graders_math = schools_data_complete[schools_data_complete["grade"] == "9th"]
ninth_graders_math_avg = ninth_graders_math.groupby("school_name")["math_score"].mean()

# Repeat the above process for the remaining grades.
tenth_graders_math = schools_data_complete[schools_data_complete["grade"] == "10th"]
tenth_graders_math_avg = tenth_graders_math.groupby("school_name")["math_score"].mean()

eleventh_graders_math = schools_data_complete[schools_data_complete["grade"] == "11th"]
eleventh_graders_math_avg = eleventh_graders_math.groupby("school_name")["math_score"].mean()

twelfth_graders_math = schools_data_complete[schools_data_complete["grade"] == "12th"]
twelfth_graders_math_avg = twelfth_graders_math.groupby("school_name")["math_score"].mean()

# Frame the variables in a new DataFrame.
grade_level_math_avg = pd.DataFrame({"9th": ninth_graders_math_avg,
                                    "10th": tenth_graders_math_avg,
                                    "11th": eleventh_graders_math_avg,
                                    "12th": twelfth_graders_math_avg})

# Simply for getting a clean DataFrame, remove the index label.
grade_level_math_avg.index.name = None
# Display the DataFrame
grade_level_math_avg


,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


# Reading Scores by Grade

In [23]:
# Tapping into the 'schools_data_complete' DataFrame, set 'grade' to 9th. 
# Group the initial variable by 'school_name' and calculate the average of 'math_score'

ninth_graders_reading = schools_data_complete[schools_data_complete["grade"] == "9th"]
ninth_graders_reading_avg = ninth_graders_reading.groupby("school_name")["reading_score"].mean()

# Repeat the above process for the remaining grades.
tenth_graders_reading = schools_data_complete[schools_data_complete["grade"] == "10th"]
tenth_graders_reading_avg = tenth_graders_reading.groupby("school_name")["reading_score"].mean()

eleventh_graders_reading = schools_data_complete[schools_data_complete["grade"] == "11th"]
eleventh_graders_reading_avg = eleventh_graders_reading.groupby("school_name")["reading_score"].mean()

twelfth_graders_reading = schools_data_complete[schools_data_complete["grade"] == "12th"]
twelfth_graders_reading_avg = twelfth_graders_reading.groupby("school_name")["reading_score"].mean()

# Frame the variables in a new DataFrame.
grade_level_reading_avg = pd.DataFrame({"9th": ninth_graders_reading_avg,
                                    "10th": tenth_graders_reading_avg,
                                    "11th": eleventh_graders_reading_avg,
                                    "12th": twelfth_graders_reading_avg})

# Simply for getting a clean DataFrame, remove the index label.
grade_level_reading_avg.index.name = None
# Display the DataFrame
grade_level_reading_avg


,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


# Scores by School Spending 

In [24]:
# Creating bins and labels
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Creating a new DataFrame by coping the 'school_summary' DataFrame
school_spending_df = school_summary.copy()

# Convert the values in 'Per Student Budget' back to an integer as it is a formatted string
school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].str.replace('[^\d.]', '', regex=True)
school_spending_df["Per Student Budget"] = pd.to_numeric(school_spending_df["Per Student Budget"])

# Using the pd.cut() method to create a new categorization column
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"],
                                                            bins=spending_bins, labels=labels)

school_spending_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
Bailey High School,District,4976,"$3,124,928.00",628.0,77.048432,81.033963,66.680064,81.933280,54.642283,$585-630
Cabrera High School,Charter,1858,"$1,081,356.00",582.0,83.061895,83.975780,94.133477,97.039828,91.334769,<$585
Figueroa High School,District,2949,"$1,884,411.00",639.0,76.711767,81.158020,65.988471,80.739234,53.204476,$630-645
Ford High School,District,2739,"$1,763,916.00",644.0,77.102592,80.746258,68.309602,79.299014,54.289887,$630-645
Griffin High School,Charter,1468,"$917,500.00",625.0,83.351499,83.816757,93.392371,97.138965,90.599455,$585-630
Hernandez High School,District,4635,"$3,022,020.00",652.0,77.289752,80.934412,66.752967,80.862999,53.527508,$645-680
Holden High School,Charter,427,"$248,087.00",581.0,83.803279,83.814988,92.505855,96.252927,89.227166,<$585
Huang High School,District,2917,"$1,910,635.00",655.0,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680
Johnson High School,District,4761,"$3,094,650.00",650.0,77.072464,80.966394,66.057551,81.222432,53.539172,$645-680
Pena High School,Charter,962,"$585,858.00",609.0,83.839917,84.044699,94.594595,95.945946,90.540541,$585-630


In [25]:
# Averages for the desired columns based on 'Spending Ranges (Per Student)'
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
spending_overall_passing = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

# Creating a new DataFrame from these new variables
spending_summary = pd.DataFrame({"Average Math Score": spending_math_scores,
                                "Average Reading Score": spending_reading_scores,
                                "% Passing Math": spending_passing_math,
                                "% Passing Reading": spending_passing_reading,
                                "% Overall Passing": spending_overall_passing})
# Display the new DataFrame
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


# Scores by School Size

In [26]:
# Creating bins and labels
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Using the pd.cut() method to create a new categorization column
school_summary["School Size"] = pd.cut(school_summary["Total Students"], bins=size_bins, labels=labels)
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,Small (<1000)
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,Large (2000-5000)
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541,Small (<1000)


In [27]:
# Averages for the desired columns based on 'School Size'
size_math_scores = school_summary.groupby("School Size")["Average Math Score"].mean()
size_reading_scores = school_summary.groupby("School Size")["Average Reading Score"].mean()
size_passing_math = school_summary.groupby("School Size")["% Passing Math"].mean()
size_passing_reading = school_summary.groupby("School Size")["% Passing Reading"].mean()
size_passing_overall = school_summary.groupby("School Size")["% Overall Passing"].mean()

# Creating a new DataFrame from these variables
size_summary = pd.DataFrame({"Average Math Score": size_math_scores,
                            "Average Reading Score": size_reading_scores,
                            "% Passing Math": size_passing_math,
                            "% Passing Reading": size_passing_reading,
                            "% Overall Passing": size_passing_overall})
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


# Scores by School Type

In [28]:
# Averages for the desired columns based on 'School Type'
average_math_score_by_type = school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = school_summary.groupby(["School Type"])["% Overall Passing"].mean()

# Creating a new DataFrame from these variables
type_summary = pd.DataFrame({"Average Math Score": average_math_score_by_type,
                            "Average Reading Score": average_reading_score_by_type,
                            "% Passing Math": average_percent_passing_math_by_type,
                            "% Passing Reading": average_percent_passing_reading_by_type,
                            "% Overall Passing": average_percent_overall_passing_by_type})
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
